In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
def load_images_from_folder(folder):
    images = []
    global di
    di = {}
    
    for i in os.listdir(folder) :
        subfolder = folder+"/"+i
        di[i] = []
        for j in os.listdir(subfolder):
            new_path = subfolder+"/"+j
            img = cv2.imread(new_path)
            if type(img) != None:
                images.append(img)
                di[i].append(img)

    # labelling the images classes with numeric values
    global plant_labels
    plant_labels = {}
    cnt = 0
    for i in di:
        plant_labels[i] = cnt
        cnt += 1

    return images

In [3]:
def create_x_y(resized_image_dir):
    global x , y
    x , y = [] , []
    for name , images in di.items():
        print(name , len(images))
        for image in range(len(images)):
            try:
                resized_image = cv2.resize(images[image] , (224,224), interpolation=cv2.INTER_AREA)
                x.append(resized_image)
                y.append(plant_labels[name])
                cv2.imwrite(f"{resized_image_dir}/{name}_{image}.jpg", resized_image)
            except:
                print(name , image , "error")
    x = np.array(x)
    y = np.array(y)

In [4]:
def model(flatten_function , optimizer_function , x , y):
    num_class = 5
    from keras.api._v2.keras import metrics
    model = Sequential ([
        layers.Conv2D(16,3 , padding='same' , activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32 ,3 , padding='same' , activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64 ,3 , padding='same' , activation='relu'),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128 , activation=flatten_function),
        layers.Dense(num_class)
    ])

    model.compile(optimizer = optimizer_function,
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics = ['accuracy'])

    model.fit(x , y , epochs=30)

    return model

In [5]:
dataset1 = load_images_from_folder("C:/Users/SANYA CHANDEL/Desktop/weeds_dataset")

In [6]:
# Please give the path of dataset as follows:  # path = "C:/Users/gmaha/Downloads/dataset"

create_x_y("C:/Users/SANYA CHANDEL/Desktop/resized_images")
di1 = di
x1 , y1 = x , y

broadleaf 52
grass 58
soil 50
soybean 56


In [7]:
x_train1 , x_test1 , y_train1 , y_test1 = train_test_split(x , y , random_state=42)

x_train_sc1 = x_train1 / 255
x_test_sc1 = x_test1 / 255

# model with relu and adam

In [9]:
model1 = model('relu' , 'adam' , x_train_sc1 , y_train1)

Epoch 1/30
6/6 [==============================] - 7s 673ms/step - loss: 2.1510 - accuracy: 0.2901
Epoch 2/30
6/6 [==============================] - 4s 670ms/step - loss: 1.1978 - accuracy: 0.4630
Epoch 3/30
6/6 [==============================] - 4s 632ms/step - loss: 0.8866 - accuracy: 0.5864
Epoch 4/30
6/6 [==============================] - 4s 661ms/step - loss: 0.7143 - accuracy: 0.6420
Epoch 5/30
6/6 [==============================] - 4s 655ms/step - loss: 0.5584 - accuracy: 0.7407
Epoch 6/30
6/6 [==============================] - 4s 646ms/step - loss: 0.3807 - accuracy: 0.9074
Epoch 7/30
6/6 [==============================] - 4s 667ms/step - loss: 0.2676 - accuracy: 0.9259
Epoch 8/30
6/6 [==============================] - 4s 635ms/step - loss: 0.2281 - accuracy: 0.9259
Epoch 9/30
6/6 [==============================] - 4s 628ms/step - loss: 0.2866 - accuracy: 0.8889
Epoch 10/30
6/6 [==============================] - 4s 643ms/step - loss: 0.1712 - accuracy: 0.9383
Epoch 11/30
6/6 [==

In [10]:
model1.evaluate(x_test_sc1 , y_test1)

predictions1 = model1.predict(x_test_sc1)
pre1 = []
for i in range(len(predictions1)):
    score = tf.nn.softmax(predictions1[i])
    pre1.append(np.argmax(score))

y_test1[:5] , pre1[:5]

2/2 [==============================] - 1s 139ms/step


(array([3, 3, 2, 3, 0]), [3, 3, 2, 1, 0])

# model with softmax and adam

In [11]:
model2 = model('softmax' , 'adam' , x_train_sc1 , y_train1)

Epoch 1/30
6/6 [==============================] - 6s 681ms/step - loss: 1.5891 - accuracy: 0.2099
Epoch 2/30
6/6 [==============================] - 4s 710ms/step - loss: 1.5594 - accuracy: 0.2716
Epoch 3/30
6/6 [==============================] - 4s 650ms/step - loss: 1.5468 - accuracy: 0.2716
Epoch 4/30
6/6 [==============================] - 4s 668ms/step - loss: 1.5459 - accuracy: 0.2716
Epoch 5/30
6/6 [==============================] - 4s 639ms/step - loss: 1.5214 - accuracy: 0.2716
Epoch 6/30
6/6 [==============================] - 4s 648ms/step - loss: 1.5210 - accuracy: 0.2716
Epoch 7/30
6/6 [==============================] - 4s 666ms/step - loss: 1.5079 - accuracy: 0.3148
Epoch 8/30
6/6 [==============================] - 4s 653ms/step - loss: 1.5020 - accuracy: 0.4815
Epoch 9/30
6/6 [==============================] - 4s 648ms/step - loss: 1.4846 - accuracy: 0.4630
Epoch 10/30
6/6 [==============================] - 4s 656ms/step - loss: 1.4722 - accuracy: 0.5556
Epoch 11/30
6/6 [==

In [12]:
model2.evaluate(x_test_sc1 , y_test1)

predictions2 = model2.predict(x_test_sc1)
pre2 = []
for i in range(len(predictions2)):
    score = tf.nn.softmax(predictions2[i])
    pre2.append(np.argmax(score))

y_test1[:5] , pre2[:5]

2/2 [==============================] - 0s 89ms/step


(array([3, 3, 2, 3, 0]), [3, 1, 2, 3, 1])


# model with softmax and rmsprop

In [13]:
model3 = model('softmax' , 'rmsprop' , x_train_sc1 , y_train1)


Epoch 1/30
6/6 [==============================] - 4s 502ms/step - loss: 1.5898 - accuracy: 0.1667
Epoch 2/30
6/6 [==============================] - 3s 545ms/step - loss: 1.5603 - accuracy: 0.2284
Epoch 3/30
6/6 [==============================] - 3s 546ms/step - loss: 1.5539 - accuracy: 0.2407
Epoch 4/30
6/6 [==============================] - 3s 465ms/step - loss: 1.5769 - accuracy: 0.2716
Epoch 5/30
6/6 [==============================] - 3s 452ms/step - loss: 1.5730 - accuracy: 0.2716
Epoch 6/30
6/6 [==============================] - 3s 492ms/step - loss: 1.5696 - accuracy: 0.2716
Epoch 7/30
6/6 [==============================] - 3s 501ms/step - loss: 1.5667 - accuracy: 0.2716
Epoch 8/30
6/6 [==============================] - 3s 475ms/step - loss: 1.5635 - accuracy: 0.2716
Epoch 9/30
6/6 [==============================] - 3s 470ms/step - loss: 1.5608 - accuracy: 0.2716
Epoch 10/30
6/6 [==============================] - 3s 484ms/step - loss: 1.5583 - accuracy: 0.2716
Epoch 11/30
6/6 [==

In [14]:
model3.evaluate(x_test_sc1 , y_test1)

predictions3 = model3.predict(x_test_sc1)
pre3 = []
for i in range(len(predictions3)):
    score = tf.nn.softmax(predictions3[i])
    pre3.append(np.argmax(score))

y_test1[:5] , pre3[:5]

2/2 [==============================] - 0s 76ms/step


(array([3, 3, 2, 3, 0]), [1, 1, 1, 1, 1])

In [15]:
maxa = 0
for i in di1:
    maxa = max(maxa , len(di1[i]))
maxa

58

# data preprocessing starts

In [17]:
path =  "C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/resized_images"
for key in di:
    os.mkdir(path + '/' + key)

In [18]:
datagen = ImageDataGenerator(
    rotation_range= 40,
    width_shift_range = 0.2,
    height_shift_range= 0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip = True,
    vertical_flip = True,
    featurewise_center =True,
    featurewise_std_normalization = True,
    fill_mode = 'nearest'
)

for name in di:
    
    images = di[name]    #It will access the keys that is the classes present in dictionary one bu one and store all the images presnt in it in form of a list.
    dir = "C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/resized_images/" + name + "/"  # specifying the diractory path where to store the data
    print(name , len(images))

    for i in range(len(images)): # iterating all the images one by one
        cnt = 0
        filename = dir + '/' + 'gray-' + name +"_" +str(i) + '.jpeg'
        try:
            gray_image = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
            cv2.imwrite(filename, gray_image)
        except:
            print(name , i , "gray error")
        try:
            new_img = images[i].reshape((1,) + images[i].shape) #reshaping the new_img from (x,y,3) to (1,x,y,3)
            for batch in datagen.flow(new_img , batch_size = 1, save_to_dir= dir , save_prefix= filename.split('.')[0]  , save_format='jpeg'):
                    cnt += 1
                    if cnt == 4:
                        break
        except:
            print(name , "error")

broadleaf 52


C:\Users\SANYA CHANDEL\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
C:\Users\SANYA CHANDEL\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


grass 58
soil 50
soybean 56


In [19]:
augumented_data = load_images_from_folder("C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/resized_images")

In [21]:
path =  "C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images"
for key in di:
    os.mkdir(path + '/' + key)

In [24]:
import os

def create_x_y2(augmented_image_dir):
    x, y = [], []

    for name, images in di.items():
        print(name, len(images))
        for image_index, image in enumerate(images):
            try:
                resized_image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
                x.append(resized_image)
                y.append(plant_labels[name])

                # Save the augmented image
                save_path = os.path.join(augmented_image_dir, name, f"{name}_{image_index}.jpg")
                cv2.imwrite(save_path, resized_image)
                print(f"Saved: {save_path}")
            except Exception as e:
                print(name, image_index, "error:", e)

    x = np.array(x)
    y = np.array(y)
    
    return x, y


In [25]:
augmented_image_dir = "C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images"
x_augmented, y_augmented = create_x_y2(augmented_image_dir)


broadleaf 260
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_0.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_1.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_2.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_3.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_4.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_5.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_6.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_7.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\bro

Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_86.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_87.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_88.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_89.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_90.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_91.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_92.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_93.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf

Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_185.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_186.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_187.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_188.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_189.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_190.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_191.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\broadleaf_192.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\broadleaf\b

Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_12.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_13.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_14.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_15.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_16.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_17.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_18.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_19.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_20.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augm

Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_96.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_97.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_98.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_99.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_100.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_101.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_102.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_103.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_104.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset

Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_232.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_233.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_234.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_235.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_236.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_237.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_238.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_239.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\grass\grass_240.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dat

Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_79.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_80.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_81.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_82.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_83.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_84.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_85.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_86.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_87.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_imag

Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_221.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_222.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_223.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_224.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_225.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_226.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_227.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_228.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soil\soil_229.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_res

Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_60.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_61.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_62.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_63.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_64.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_65.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_66.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_67.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_68.jpg
Saved: C:/Users/SANYA CHANDE

Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_155.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_156.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_157.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_158.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_159.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_160.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_161.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_162.jpg
Saved: C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images\soybean\soybean_163.jpg
Saved: C:/Users/SAN

In [34]:
import os
import shutil

# Source folders
original_images_folder = "C:/Users/SANYA CHANDEL/Desktop/weeds_dataset"
augmented_images_folder = "C:/Users/SANYA CHANDEL/Desktop/weeds_project/dataset/augmented_resized_images"

# Destination folder on the desktop
combined_folder = "C:/Users/SANYA CHANDEL/Desktop/combined_images"

# Create the combined folder if it doesn't exist
os.makedirs(combined_folder, exist_ok=True)

# Function to copy files and maintain folder structure
def copy_files(source_folder, destination_folder):
    for root, _, files in os.walk(source_folder):
        relative_path = os.path.relpath(root, source_folder)
        destination_path = os.path.join(destination_folder, relative_path)
        os.makedirs(destination_path, exist_ok=True)

        for file in files:
            source_file_path = os.path.join(root, file)
            destination_file_path = os.path.join(destination_path, file)

            try:
                shutil.copy2(source_file_path, destination_file_path)
            except Exception as e:
                print(f"Error copying {file}: {e}")

# Copy from the original images folder
copy_files(original_images_folder, combined_folder)

# Copy from the augmented images folder
copy_files(augmented_images_folder, combined_folder)

print("Images from 'weeds_dataset' and 'augmented_resized_images' added to 'combined_images' folder.")


Images from 'weeds_dataset' and 'augmented_resized_images' added to 'combined_images' folder.


In [37]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Source folder
combined_folder = "C:/Users/SANYA CHANDEL/Desktop/combined_images"

# Destination folder (augmented_data)
augmented_data_folder = "C:/Users/SANYA CHANDEL/Desktop/augmented_data"

# Create the augmented_data folder if it doesn't exist
os.makedirs(augmented_data_folder, exist_ok=True)

# Destination subfolders (train_images and test_images)
train_folder = os.path.join(augmented_data_folder, "train_images")
test_folder = os.path.join(augmented_data_folder, "test_images")

# Create the train and test folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Function to copy files and maintain folder structure
def copy_files(source_files, destination_folder):
    for source_file_path in source_files:
        relative_path = os.path.relpath(source_file_path, combined_folder)
        destination_file_path = os.path.join(destination_folder, relative_path)

        try:
            os.makedirs(os.path.dirname(destination_file_path), exist_ok=True)
            shutil.copy2(source_file_path, destination_file_path)
        except Exception as e:
            print(f"Error copying {source_file_path}: {e}")

# Split into train and test
all_files = []
for root, _, files in os.walk(combined_folder):
    all_files.extend([os.path.join(root, file) for file in files])

train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)

# Copy files to train folder
copy_files(train_files, train_folder)

# Copy files to test folder
copy_files(test_files, test_folder)

print("Images split into train and test folders within 'augmented_data' folder.")


Images split into train and test folders within 'augmented_data' folder.
